In [196]:
import numpy as np 
import gurobipy as gp
from gurobipy import GRB
import random

Q_csv = np.loadtxt(open("q_matrices/dim75_Q00.csv", "rb"), delimiter=",", skiprows=0)
length = Q_csv.shape[0]
print(length)

def fill_diagonals(csv_matrix, num_diagonals): 
    Q = np.zeros((len(csv_matrix), len(csv_matrix)))
    for i in range(num_diagonals):
        for j in range(len(csv_matrix) - i):
            Q[j,j+i] = csv_matrix[j,j+i]
            Q[j+i,j] = csv_matrix[j+i,j]
    return Q

def fill_rows_columns(csv_matrix, num_rows):
    Q = np.zeros((len(csv_matrix), len(csv_matrix)))
    for i in range(num_rows):
        Q[i,:] = csv_matrix[i,:]
        Q[:,i] = csv_matrix[:,i]
    return Q    

def calculate_sparsity(matrix):
    return np.count_nonzero(matrix)/(matrix.shape[0]*matrix.shape[1])

Q = fill_rows_columns(Q_csv, 48)
print(calculate_sparsity(Q))
print(Q)

75
0.8704
[[-1458498.5    14500.     15000.  ...     8500.      8500.      8500. ]
 [   14500.  -1284998.5    14000.  ...     6000.      6000.      6000. ]
 [   15000.     14000.  -1319498.5 ...     6500.      6500.      6500. ]
 ...
 [    8500.      6000.      6500.  ...        0.         0.         0. ]
 [    8500.      6000.      6500.  ...        0.         0.         0. ]
 [    8500.      6000.      6500.  ...        0.         0.         0. ]]


In [197]:
env = gp.Env(empty=True)
env.setParam("LogToConsole", 1)
env.setParam("LogFile", 'gurobi.txt')
env.start()

problem = gp.Model(env=env)

x = problem.addMVar(len(Q), vtype=GRB.BINARY, name = 'x')
obj = x@Q@x
problem.setObjective(obj, GRB.MINIMIZE)
problem.params.NonConvex = 2
problem.optimize()

sol_x = []
for v in problem.getVars():
    if 'x' in v.varName:
        sol_x.append(v.x)

Set parameter Username
Set parameter LogFile to value "gurobi.txt"
Academic license - for non-commercial use only - expires 2024-06-06
Set parameter NonConvex to value 2
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i7-8559U CPU @ 2.70GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 75 columns and 0 nonzeros
Model fingerprint: 0x5f4be6b3
Model has 2472 quadratic objective terms
Variable types: 0 continuous, 75 integer (75 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+04, 3e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 0.0000000
Found heuristic solution: objective -8842928.000
Found heuristic solution: objective -3.13389e+07
Presolve removed 0 rows and 30 columns
Presolve time: 0.01s
Presolved: 0 rows, 45 columns, 0 nonzeros
Presolved model ha

In [198]:
x = np.asarray(sol_x)
print('Objective')
print(x@Q@x)

Objective
-31338931.0


In [199]:
import dimod
from dwave.system.samplers import DWaveSampler, LeapHybridSampler, DWaveCliqueSampler
from dwave.system.composites import EmbeddingComposite, AutoEmbeddingComposite, ReverseBatchStatesComposite

def matrix_to_qubo_helper(A):
    Q = {}
    for i in range(A.shape[1]):
        for j in range(i, A.shape[1]):
            if i != j:
                Q[(i,j)] = A[i][j] + A[j][i]
            else:
                Q[(i,j)] = A[i][j]
    return Q

In [ ]:
bqm = dimod.BinaryQuadraticModel.from_qubo(matrix_to_qubo_helper(Q))
sampler = AutoEmbeddingComposite(DWaveSampler(profile='kadysong@berkeley.edu'))
sample_set = sampler.sample(bqm, num_reads = 1000)
print(sample_set.info["timing"]) 

In [ ]:
# Use the first solution
x = list(sample_set.samples()[0].values())
print('Objective')
print(x@Q@x)